In [ ]:
# Імпорт бібліотек
import os
from typing import TypedDict, Annotated, Literal
from typing_extensions import TypedDict
import operator

from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, SystemMessage
import pandas as pd
import numpy as np
from datetime import datetime, date
import json

from api_config import get_openai_client, get_model_config, check_api_keys, print_config_status


try:
    from IPython.display import Image, display
    import matplotlib.pyplot as plt
    import networkx as nx
except ImportError:
    pass


print_config_status()




In [ ]:
# Стан містить всю інформацію, яка передається між вузлами графа

class AgentState(TypedDict):
    """Стан агента - містить всі дані для прийняття рішень"""
    
    # Вхідні дані
    input_data: dict  # Початкові дані про стан енергосистеми
    timestamp: str   # Час обробки
    
    # Результати валідації
    data_quality: dict  # Оцінка якості даних: {"score": float, "issues": list}
    is_valid: bool      # Чи дані валідні для подальшої обробки
    
    # Оцінка ризиків
    risk_assessment: dict  # {"level": str, "score": float, "regions": list}
    
    # Сценарії та рішення
    scenarios: list      # Список альтернативних сценаріїв дій
    selected_scenario: dict  # Вибраний сценарій
    decision_confidence: float  # Впевненість у рішенні (0-1)
    
    # Історія та контекст
    decision_history: list  # Історія прийнятих рішень
    uncertainty_flags: list  # Прапорці невизначеності
    retry_count: int  # Кількість спроб повернення назад
    
    # Результат
    action: dict  # Фінальна дія для виконання
    reasoning: str  # Пояснення прийнятого рішення
    
    # Метадані
    current_node: str  # Поточний вузол графа
    path_taken: list   # Шлях пройдених вузлів


In [ ]:

def validate_data(state: AgentState) -> AgentState:
    """
    Вузол 1: Валідація вхідних даних
    Перевіряє наявність, консистентність та якість даних
    """
    print(f"[validate_data] Перевірка якості даних...")
    
    input_data = state.get("input_data", {})
    issues = []
    score = 1.0
    
    # Перевірка наявності ключових полів
    required_fields = ["region", "date", "alerts", "is_damaged"]
    for field in required_fields:
        if field not in input_data:
            issues.append(f"Відсутнє поле: {field}")
            score -= 0.2
    
    # Перевірка типів даних
    if "alerts" in input_data:
        if not isinstance(input_data["alerts"], (int, float)) or input_data["alerts"] < 0:
            issues.append("Невірний формат alerts (має бути невід'ємне число)")
            score -= 0.1
    
    if "is_damaged" in input_data:
        if not isinstance(input_data["is_damaged"], bool):
            issues.append("Невірний формат is_damaged (має бути boolean)")
            score -= 0.1
    
    # Перевірка на NaN/None значення
    for key, value in input_data.items():
        if value is None or (isinstance(value, float) and np.isnan(value)):
            issues.append(f"Значення None/NaN у полі: {key}")
            score -= 0.1
    
    score = max(0.0, score)  # Не менше 0
    
    state["data_quality"] = {
        "score": score,
        "issues": issues,
        "timestamp": state.get("timestamp", datetime.now().isoformat())
    }
    state["is_valid"] = score >= 0.5  # Поріг валідності
    state["current_node"] = "validate_data"
    state["path_taken"] = state.get("path_taken", []) + ["validate_data"]
    
    print(f"[validate_data] Якість даних: {score:.2f}, Валідність: {state['is_valid']}")
    
    return state


In [ ]:
def check_consistency(state: AgentState) -> AgentState:
    """
    Вузол 2: Перевірка логічної консистентності даних
    Виявляє суперечності між різними джерелами даних
    """
    print(f"[check_consistency] Перевірка консистентності...")
    
    input_data = state.get("input_data", {})
    inconsistencies = []
    
    # Перевірка логічних суперечностей
    alerts = input_data.get("alerts", 0)
    is_damaged = input_data.get("is_damaged", False)
    blackout_consumers = input_data.get("blackout_consumers", 0)
    
    # Якщо є пошкодження, але немає тривог - можлива суперечність
    if is_damaged and alerts == 0:
        inconsistencies.append(
            "Пошкодження зафіксовано без тривог - можлива затримка даних"
        )
    
    # Якщо є відключення, але немає тривог і пошкоджень
    if blackout_consumers > 0 and alerts == 0 and not is_damaged:
        inconsistencies.append(
            "Відключення без тривог/пошкоджень - можливий технічний збій"
        )
    
    # Перевірка погодних даних
    weather = input_data.get("weather", {})
    if weather:
        wind_speed = weather.get("wind_speed_max", 0)
        if wind_speed > 25 and not is_damaged and alerts == 0:
            inconsistencies.append(
                "Сильний вітер без тривог/пошкоджень - можлива невідповідність"
            )
    
    # Оновлюємо прапорці невизначеності
    uncertainty_flags = state.get("uncertainty_flags", [])
    if inconsistencies:
        uncertainty_flags.append({
            "type": "data_inconsistency",
            "issues": inconsistencies,
            "timestamp": datetime.now().isoformat()
        })
    
    state["uncertainty_flags"] = uncertainty_flags
    state["data_quality"]["inconsistencies"] = inconsistencies
    state["current_node"] = "check_consistency"
    state["path_taken"] = state.get("path_taken", []) + ["check_consistency"]
    
    print(f"[check_consistency] Виявлено суперечностей: {len(inconsistencies)}")
    
    return state


In [ ]:
def assess_risks(state: AgentState) -> AgentState:
    """
    Вузол 3: Оцінка ризиків для енергосистеми
    Аналізує поточний стан та визначає рівень ризику
    """
    print(f"[assess_risks] Оцінка ризиків...")
    
    input_data = state.get("input_data", {})
    risk_score = 0.0
    risk_factors = []
    
    # Фактор 1: Тривоги
    alerts = input_data.get("alerts", 0)
    if alerts > 0:
        risk_score += min(alerts * 0.1, 0.4)  # Максимум 0.4
        risk_factors.append(f"Активні тривоги: {alerts}")
    
    # Фактор 2: Пошкодження
    is_damaged = input_data.get("is_damaged", False)
    if is_damaged:
        risk_score += 0.3
        risk_factors.append("Зафіксовано пошкодження інфраструктури")
    
    # Фактор 3: Відключення
    blackout_consumers = input_data.get("blackout_consumers", 0)
    if blackout_consumers > 0:
        # Нормалізуємо: 1000+ споживачів = високий ризик
        normalized_blackout = min(blackout_consumers / 10000.0, 1.0)
        risk_score += normalized_blackout * 0.3
        risk_factors.append(f"Відключення: {blackout_consumers} споживачів")
    
    # Фактор 4: Погодні умови
    weather = input_data.get("weather", {})
    if weather:
        wind_speed = weather.get("wind_speed_max", 0)
        if wind_speed > 20:
            risk_score += min((wind_speed - 20) / 30.0, 0.2)
            risk_factors.append(f"Сильний вітер: {wind_speed} км/год")
        
        precipitation = weather.get("precipitation", 0)
        if precipitation > 20:
            risk_score += min(precipitation / 100.0, 0.1)
            risk_factors.append(f"Сильні опади: {precipitation} мм")
    
    # Визначаємо рівень ризику
    if risk_score >= 0.7:
        risk_level = "критичний"
    elif risk_score >= 0.4:
        risk_level = "високий"
    elif risk_score >= 0.2:
        risk_level = "середній"
    else:
        risk_level = "низький"
    
    state["risk_assessment"] = {
        "level": risk_level,
        "score": min(risk_score, 1.0),
        "factors": risk_factors,
        "regions": [input_data.get("region", "невідомо")],
        "timestamp": datetime.now().isoformat()
    }
    state["current_node"] = "assess_risks"
    state["path_taken"] = state.get("path_taken", []) + ["assess_risks"]
    
    print(f"[assess_risks] Рівень ризику: {risk_level} ({risk_score:.2f})")
    
    return state


In [ ]:
def generate_scenarios(state: AgentState) -> AgentState:
    """
    Вузол 4: Генерація альтернативних сценаріїв дій
    Створює кілька можливих стратегій реагування
    """
    print(f"[generate_scenarios] Генерація сценаріїв...")
    
    risk_assessment = state.get("risk_assessment", {})
    risk_level = risk_assessment.get("level", "низький")
    input_data = state.get("input_data", {})
    region = input_data.get("region", "невідомо")
    
    scenarios = []
    
    # Сценарій 1: Консервативний (мінімальні дії)
    scenarios.append({
        "id": "conservative",
        "name": "Консервативний підхід",
        "actions": [
            "Моніторинг ситуації",
            "Підготовка резервних ресурсів"
        ],
        "expected_impact": "низький",
        "cost": "низький",
        "risk_reduction": 0.1,
        "suitable_for": ["низький", "середній"]
    })
    
    # Сценарій 2: Активний (середні дії)
    scenarios.append({
        "id": "active",
        "name": "Активне реагування",
        "actions": [
            "Підвищення готовності бригад",
            "Перерозподіл навантаження",
            "Активація резервних потужностей"
        ],
        "expected_impact": "середній",
        "cost": "середній",
        "risk_reduction": 0.3,
        "suitable_for": ["середній", "високий"]
    })
    
    # Сценарій 3: Аварійний (максимальні дії)
    scenarios.append({
        "id": "emergency",
        "name": "Аварійний режим",
        "actions": [
            "Мобілізація всіх ресурсів",
            "Екстрене відновлення пошкоджень",
            "Координація з місцевими службами",
            "Повідомлення населення"
        ],
        "expected_impact": "високий",
        "cost": "високий",
        "risk_reduction": 0.6,
        "suitable_for": ["високий", "критичний"]
    })
    
    # Сценарій 4: Проактивний (запобіжні заходи)
    scenarios.append({
        "id": "proactive",
        "name": "Проактивні заходи",
        "actions": [
            "Попередження про можливі проблеми",
            "Профілактичний огляд обладнання",
            "Підготовка до можливих змін погоди"
        ],
        "expected_impact": "середній",
        "cost": "низький",
        "risk_reduction": 0.2,
        "suitable_for": ["низький", "середній"]
    })
    
    state["scenarios"] = scenarios
    state["current_node"] = "generate_scenarios"
    state["path_taken"] = state.get("path_taken", []) + ["generate_scenarios"]
    
    print(f"[generate_scenarios] Згенеровано {len(scenarios)} сценаріїв")
    
    return state


In [ ]:


def compare_scenarios(state: AgentState) -> AgentState:
    """
    Вузол 5: Порівняння альтернативних сценаріїв
    Оцінює кожен сценарій та визначає найкращий
    """
    print(f"[compare_scenarios] Порівняння сценаріїв...")
    
    scenarios = state.get("scenarios", [])
    risk_assessment = state.get("risk_assessment", {})
    risk_level = risk_assessment.get("level", "низький")
    
    if not scenarios:
        state["selected_scenario"] = None
        state["decision_confidence"] = 0.0
        return state
    
    # Оцінюємо кожен сценарій
    scored_scenarios = []
    for scenario in scenarios:
        score = 0.0
        
        # Відповідність рівню ризику
        if risk_level in scenario.get("suitable_for", []):
            score += 0.4
        
        # Ефективність зниження ризику
        risk_reduction = scenario.get("risk_reduction", 0.0)
        score += risk_reduction * 0.3
        
        # Вартість (нижча = краще)
        cost = scenario.get("cost", "середній")
        cost_scores = {"низький": 0.2, "середній": 0.1, "високий": 0.0}
        score += cost_scores.get(cost, 0.1)
        
        # Враховуємо невизначеність
        uncertainty_flags = state.get("uncertainty_flags", [])
        if uncertainty_flags:
            # При невизначеності краще консервативний підхід
            if scenario["id"] == "conservative":
                score += 0.1
        
        scored_scenarios.append({
            **scenario,
            "total_score": score
        })
    
    # Сортуємо за оцінкою
    scored_scenarios.sort(key=lambda x: x["total_score"], reverse=True)
    
    # Вибираємо найкращий сценарій
    best_scenario = scored_scenarios[0] if scored_scenarios else None
    
    # Визначаємо впевненість у рішенні
    if len(scored_scenarios) > 1:
        best_score = scored_scenarios[0]["total_score"]
        second_score = scored_scenarios[1]["total_score"]
        # Чим більша різниця, тим вища впевненість
        confidence = min((best_score - second_score) * 2, 1.0)
    else:
        confidence = 0.5
    
    state["scenarios"] = scored_scenarios  # Оновлюємо з оцінками
    state["selected_scenario"] = best_scenario
    state["decision_confidence"] = confidence
    state["current_node"] = "compare_scenarios"
    state["path_taken"] = state.get("path_taken", []) + ["compare_scenarios"]
    
    if best_scenario:
        print(f"[compare_scenarios] Обрано сценарій: {best_scenario['name']} (впевненість: {confidence:.2f})")
    
    return state


In [ ]:
# ============================================
# ВУЗЛИ ГРАФА: ВАЛІДАЦІЯ РІШЕННЯ
# ============================================

def validate_decision(state: AgentState) -> AgentState:
    """
    Вузол 6: Перевірка валідності прийнятого рішення
    Може повернути назад, якщо рішення невпевнене або суперечливе
    """
    print(f"[validate_decision] Валідація рішення...")
    
    decision_confidence = state.get("decision_confidence", 0.0)
    selected_scenario = state.get("selected_scenario")
    uncertainty_flags = state.get("uncertainty_flags", [])
    retry_count = state.get("retry_count", 0)
    
    is_valid = True
    issues = []
    
    # Перевірка 1: Чи є обраний сценарій
    if not selected_scenario:
        is_valid = False
        issues.append("Сценарій не обрано")
    
    # Перевірка 2: Впевненість у рішенні
    if decision_confidence < 0.3:
        is_valid = False
        issues.append(f"Низька впевненість у рішенні: {decision_confidence:.2f}")
    
    # Перевірка 3: Наявність критичних невизначеностей
    critical_uncertainties = [
        flag for flag in uncertainty_flags 
        if flag.get("type") == "data_inconsistency" and len(flag.get("issues", [])) > 2
    ]
    if critical_uncertainties and retry_count < 2:
        is_valid = False
        issues.append("Критичні невизначеності в даних")
    
    # Перевірка 4: Максимальна кількість спроб
    if retry_count >= 3:
        is_valid = True  # Примусово приймаємо рішення після 3 спроб
        issues.append("Досягнуто максимум спроб - приймаємо рішення")
    
    state["decision_validation"] = {
        "is_valid": is_valid,
        "issues": issues,
        "retry_count": retry_count
    }
    state["current_node"] = "validate_decision"
    state["path_taken"] = state.get("path_taken", []) + ["validate_decision"]
    
    print(f"[validate_decision] Валідність: {is_valid}, Проб: {retry_count}")
    
    return state


In [ ]:
# ============================================
# ВУЗЛИ ГРАФА: ВИКОНАННЯ ДІЇ
# ============================================

def execute_action(state: AgentState) -> AgentState:
    """
    Вузол 7: Виконання фінальної дії
    Формує конкретні інструкції для виконання
    """
    print(f"[execute_action] Формування дії...")
    
    selected_scenario = state.get("selected_scenario", {})
    input_data = state.get("input_data", {})
    risk_assessment = state.get("risk_assessment", {})
    
    # Формуємо фінальну дію
    action = {
        "type": "energy_system_response",
        "scenario_id": selected_scenario.get("id", "unknown"),
        "scenario_name": selected_scenario.get("name", "Невідомо"),
        "actions": selected_scenario.get("actions", []),
        "region": input_data.get("region", "невідомо"),
        "risk_level": risk_assessment.get("level", "невідомо"),
        "timestamp": datetime.now().isoformat(),
        "priority": "high" if risk_assessment.get("level") in ["високий", "критичний"] else "normal"
    }
    
    # Формуємо пояснення
    reasoning_parts = [
        f"Обрано сценарій: {selected_scenario.get('name', 'невідомо')}",
        f"Рівень ризику: {risk_assessment.get('level', 'невідомо')}",
        f"Впевненість у рішенні: {state.get('decision_confidence', 0.0):.2f}"
    ]
    
    if state.get("uncertainty_flags"):
        reasoning_parts.append(f"Враховуючи {len(state['uncertainty_flags'])} невизначеностей")
    
    reasoning = ". ".join(reasoning_parts) + "."
    
    # Оновлюємо історію рішень
    decision_history = state.get("decision_history", [])
    decision_history.append({
        "action": action,
        "reasoning": reasoning,
        "timestamp": datetime.now().isoformat(),
        "path": state.get("path_taken", [])
    })
    
    state["action"] = action
    state["reasoning"] = reasoning
    state["decision_history"] = decision_history
    state["current_node"] = "execute_action"
    state["path_taken"] = state.get("path_taken", []) + ["execute_action"]
    
    print(f"[execute_action] Дію сформовано: {action['scenario_name']}")
    
    return state


In [ ]:
# ============================================
# УМОВНІ ФУНКЦІЇ ДЛЯ ПЕРЕХОДІВ
# ============================================

def should_continue_after_validation(state: AgentState) -> Literal["assess_risks", "retry"]:
    """
    Умова: чи продовжувати після валідації даних
    """
    is_valid = state.get("is_valid", False)
    retry_count = state.get("retry_count", 0)
    
    if not is_valid and retry_count < 2:
        return "retry"
    return "assess_risks"


def should_retry_decision(state: AgentState) -> Literal["compare_scenarios", "assess_risks", "execute_action"]:
    """
    Умова: чи повертатися назад при невпевненому рішенні
    """
    validation = state.get("decision_validation", {})
    is_valid = validation.get("is_valid", True)
    retry_count = state.get("retry_count", 0)
    
    if not is_valid and retry_count < 2:
        # Повертаємося до оцінки ризиків для переоцінки
        # Лічильник спроб буде збільшено у вузлі assess_risks
        return "assess_risks"
    elif not is_valid and retry_count >= 2:
        # Після максимальної кількості спроб - приймаємо рішення
        return "execute_action"
    return "execute_action"


def should_validate_or_execute(state: AgentState) -> Literal["validate_decision", "execute_action"]:
    """
    Умова: чи валідувати рішення чи одразу виконувати
    """
    decision_confidence = state.get("decision_confidence", 0.0)
    uncertainty_flags = state.get("uncertainty_flags", [])
    
    # Якщо висока впевненість і немає невизначеностей - можна одразу виконувати
    if decision_confidence > 0.7 and not uncertainty_flags:
        return "execute_action"
    return "validate_decision"


In [ ]:
# ============================================
# ВУЗЕЛ: ПОВЕРНЕННЯ НАЗАД (RETRY)
# ============================================

def retry_with_adjustments(state: AgentState) -> AgentState:
    """
    Вузол для повторної спроби з корекціями
    Застосовує додаткові обробки даних при невдалій валідації
    """
    print(f"[retry] Повторна спроба з корекціями...")
    
    retry_count = state.get("retry_count", 0)
    input_data = state.get("input_data", {}).copy()
    
    # Застосовуємо корекції до даних
    # Наприклад, заповнюємо пропуски середніми значеннями
    if "alerts" not in input_data or input_data["alerts"] is None:
        input_data["alerts"] = 0
    
    if "is_damaged" not in input_data or input_data["is_damaged"] is None:
        input_data["is_damaged"] = False
    
    # Оновлюємо стан
    state["input_data"] = input_data
    state["retry_count"] = retry_count + 1
    state["current_node"] = "retry"
    state["path_taken"] = state.get("path_taken", []) + [f"retry_{retry_count + 1}"]
    
    print(f"[retry] Корекції застосовано, спроба {retry_count + 1}")
    
    return state


In [ ]:
# ============================================
# ПОБУДОВА ГРАФА LANGGRAPH
# ============================================

# Створюємо граф
workflow = StateGraph(AgentState)

# Додаємо вузли
workflow.add_node("validate_data", validate_data)
workflow.add_node("check_consistency", check_consistency)
workflow.add_node("assess_risks", assess_risks)
workflow.add_node("generate_scenarios", generate_scenarios)
workflow.add_node("compare_scenarios", compare_scenarios)
workflow.add_node("validate_decision", validate_decision)
workflow.add_node("execute_action", execute_action)
workflow.add_node("retry", retry_with_adjustments)

# Визначаємо початкову точку
workflow.set_entry_point("validate_data")

# Додаємо переходи
# Після валідації даних -> перевірка консистентності
workflow.add_edge("validate_data", "check_consistency")

# Після перевірки консистентності -> умовний перехід
workflow.add_conditional_edges(
    "check_consistency",
    should_continue_after_validation,
    {
        "assess_risks": "assess_risks",
        "retry": "retry"
    }
)

# Після retry -> повертаємося до валідації
workflow.add_edge("retry", "validate_data")

# Після оцінки ризиків -> генерація сценаріїв
workflow.add_edge("assess_risks", "generate_scenarios")

# Після генерації сценаріїв -> порівняння
workflow.add_edge("generate_scenarios", "compare_scenarios")

# Після порівняння -> умовний перехід (валідація чи виконання)
workflow.add_conditional_edges(
    "compare_scenarios",
    should_validate_or_execute,
    {
        "validate_decision": "validate_decision",
        "execute_action": "execute_action"
    }
)

# Після валідації рішення -> умовний перехід
workflow.add_conditional_edges(
    "validate_decision",
    should_retry_decision,
    {
        "compare_scenarios": "compare_scenarios",
        "assess_risks": "assess_risks"
    }
)

# Після виконання дії -> кінець
workflow.add_edge("execute_action", END)

# Компілюємо граф
app = workflow.compile()

print("Граф побудовано та скомпільовано!")
print("\nСтруктура графа:")
print("validate_data -> check_consistency -> [assess_risks | retry]")
print("assess_risks -> generate_scenarios -> compare_scenarios")
print("compare_scenarios -> [validate_decision | execute_action]")
print("validate_decision -> [compare_scenarios | assess_risks]")
print("execute_action -> END")


In [ ]:
# ============================================
# ПРИКЛАД ВИКОРИСТАННЯ
# ============================================

# Приклад 1: Нормальна ситуація
example_input_1 = {
    "region": "Київська",
    "date": "2024-01-15",
    "alerts": 2,
    "is_damaged": False,
    "blackout_consumers": 0,
    "weather": {
        "temperature_mean": 5.0,
        "wind_speed_max": 15.0,
        "precipitation": 5.0
    }
}

# Приклад 2: Критична ситуація
example_input_2 = {
    "region": "Харківська",
    "date": "2024-01-15",
    "alerts": 5,
    "is_damaged": True,
    "blackout_consumers": 5000,
    "weather": {
        "temperature_mean": -5.0,
        "wind_speed_max": 30.0,
        "precipitation": 25.0
    }
}

# Приклад 3: Неповні дані (для тестування retry)
example_input_3 = {
    "region": "Одеська",
    "date": "2024-01-15",
    "alerts": None,  # Відсутнє значення
    "is_damaged": None,
    "blackout_consumers": 1000
}

print("Приклади вхідних даних підготовлено")


In [ ]:
def run_agent(input_data: dict) -> dict:
    """
    Запускає агента з вхідними даними та повертає результат
    """
    # Ініціалізуємо початковий стан
    initial_state: AgentState = {
        "input_data": input_data,
        "timestamp": datetime.now().isoformat(),
        "data_quality": {},
        "is_valid": False,
        "risk_assessment": {},
        "scenarios": [],
        "selected_scenario": {},
        "decision_confidence": 0.0,
        "decision_history": [],
        "uncertainty_flags": [],
        "retry_count": 0,
        "action": {},
        "reasoning": "",
        "current_node": "",
        "path_taken": []
    }
    
    final_state = app.invoke(initial_state)
    

    print(f"Шлях: {' -> '.join(final_state.get('path_taken', []))}")
    print(f"Рівень ризику: {final_state.get('risk_assessment', {}).get('level', 'невідомо')}")
    print(f"Обраний сценарій: {final_state.get('selected_scenario', {}).get('name', 'невідомо')}")
    print(f"Впевненість: {final_state.get('decision_confidence', 0.0):.2f}")
    print(f"Пояснення: {final_state.get('reasoning', 'немає')}")
    
    return final_state

# Тестуємо на прикладі 1
print("Тестування на прикладі 1 (нормальна ситуація):")
result_1 = run_agent(example_input_1)


In [ ]:
# Тестування на прикладі 2 
print("\n\nТестування на прикладі 2 (критична ситуація):")
result_2 = run_agent(example_input_2)


In [ ]:

print("\n\nТестування на прикладі 3 (неповні дані):")
result_3 = run_agent(example_input_3)


## Налаштування API ключів

### Крок 1: Створіть файл `.env`

Створіть файл `.env` у корені проекту та додайте ваші API ключі:

```
OPENAI_API_KEY=sk-your-actual-key-here
OPENAI_MODEL=gpt-4o-mini
OPENAI_TEMPERATURE=0.7
```

### Крок 2: Встановіть залежності

```bash
pip install python-dotenv langchain-openai
```

### Крок 3: Перевірка

Запустіть наступну клітинку для перевірки конфігурації:


In [ ]:
# Перевірка наявності та валідності API ключів
try:
    # Спробуємо створити клієнт
    llm = get_openai_client()
    print("✓ OpenAI клієнт успішно створено")
    
    # Показуємо конфігурацію
    config = get_model_config()
    print(f"\nКонфігурація моделі:")
    print(f"  Модель: {config['model']}")
    print(f"  Температура: {config['temperature']}")
    
except Exception as e:
    print(f"✗ Помилка при створенні клієнта: {e}")
    print("\nПереконайтеся, що:")
    print("1. Файл .env існує у корені проекту")
    print("2. OPENAI_API_KEY встановлено у .env")
    print("3. Встановлено python-dotenv: pip install python-dotenv")
